# NBA Data Scrape

In [1]:
# Shoutout to my friends Ray Ford and James Ford for spending a whole weekend on ZOOM teaching me a faster scraping method :)

import requests
from bs4 import BeautifulSoup
import json
import csv
import logging
import re
import time
import datetime
from datetime import date
from dateutil.parser import parse

In [2]:
# Setup the csv writer
nbafile = open("NBA_data/NBA_Season_avgs.csv", "w", newline='', encoding='utf-8')
nbawriter = csv.writer(nbafile)
# set headings
nbawriter.writerow(['RK', 
                    'Team', 
                    'Year', 
                    'G', 
                    'W', 
                    'L', 
                    'Conf', 
                    'MP', 
                    'FG', 
                    'FGA', 
                    'FG%', 
                    '3P',
                    '3PA', 
                    '3P%', 
                    '2P', 
                    '2PA', 
                    '2P%', 
                    'FT', 
                    'FTA', 
                    'FT%', 
                    'ORB', 
                    'DRB', 
                    'AST', 
                    'STL', 
                    'BLK', 
                    'TOV', 
                    'PF', 
                    'PTS'])

103

## Select Start and End Year

In [3]:
# Define Year for URL
start_year = 1980
end_year = 2020

In [4]:
# Create model to scrape data

class NBAscrapy:
    runcount = 1
    rk = 1
    year = 1980
    
    # __init__ required to initialize the scrape tool
    def __init__(self):
        self.main()

    # starting URL according to start year above
    def main(self):
        url = "https://www.basketball-reference.com/leagues/NBA_"+str(start_year)+".html"
        self.getyearteamdata(url)

    # build data for per-year
    def getyearteamdata(self, url):
        
        # running through each table by rank since website uses rank in each table used in ascending order
        # rank will reset to 1 once we move to a new year
        self.rk = 1
        print("--------------year: " + str(self.year) + " |" + " row: " + str(self.runcount) + "-----------------")
        
        # get data, total_pages
        page = requests.get(url)
        page.encoding = "UTF-8"
        pagecontent = str(page.content)
        pagecontent = pagecontent.replace('<!--',"")
        pagecontent = pagecontent.replace("-->", "")
        pagecontent = pagecontent.replace("\n", "")

        # print(pagecontent)
        soup = BeautifulSoup(pagecontent, 'html.parser')
        soup.encode('utf-8')

        # format material for page.json()
        
        # data will be pulled from 'Team Per Game Stats' table (avg game data)
        Team = ""
        Year = self.year
        G = ""
        MP = ""
        FG = ""
        FGA = ""
        FGpercent = ""
        ThreeP = ""
        ThreePA = ""
        ThreePpercent = ""
        TwoP = ""
        TwoPA = ""
        TwoPpercent = ""
        FT = ""
        FTA = ""
        FTpercent = ""
        ORB = ""
        DRB = ""
        TRB = ""
        AST = ""
        STL = ""
        BLK = ""
        TOV = ""
        PF = ""
        PTS = ""
        
        # data will be pulled from 'Conference Standings' table
        W = ""
        L = ""
        Conf = ""
        E_W_teams = []

        # for testing we wanted to split between East and West
        # This required pulling from the 'Conference Standings' table
        
        # If statement to Eastern Conference
        if soup.find('table', {'id':'divs_standings_E'}) is not None:
            if soup.find('table', {'id':'divs_standings_E'}).findAll('tr', {'class': 'full_table'}) is not None:
                E_team_trs = soup.find('table', {'id':'divs_standings_E'}).findAll('tr', {'class': 'full_table'})
                nI = 0
                
                # Loop through Eastern Conference, collect wins, lossess
                for E_team_tr in E_team_trs:
                    E_team = {
                        'index': 'East', # Index the data by 'East' Conference
                        'teamname': E_team_tr.find('a').text.strip(),
                        'W': E_team_tr.find('td',{'data-stat': 'wins'}).text.strip(),
                        'L': E_team_tr.find('td',{'data-stat': 'losses'}).text.strip()
                    }
                    E_W_teams.append(E_team.copy())
                    nI += 1
                    
        # If statement to Western Conference
        if soup.find('table', {'id':'divs_standings_W'}) is not None:
            if soup.find('table', {'id':'divs_standings_W'}).findAll('tr', {'class': 'full_table'}) is not None:
                W_team_trs = soup.find('table', {'id':'divs_standings_W'}).findAll('tr', {'class': 'full_table'})
                nI = 0
                
                # Loop through Western Conference, collect wins, lossess
                for W_team_tr in W_team_trs:
                    W_team = {
                        'index': 'West', # Index the data by 'West' Conference
                        'teamname': W_team_tr.find('a').text.strip(),
                        'W': W_team_tr.find('td',{'data-stat': 'wins'}).text.strip(),
                        'L': W_team_tr.find('td',{'data-stat': 'losses'}).text.strip()
                    }
                    E_W_teams.append(W_team.copy())
                    nI += 1
                    
        # If statement to collect data from league "Team Per Game Stats" table            
        if soup.find('table', {'id':'team-stats-per_game'}) is not None:
            if soup.find('table', {'id':'team-stats-per_game'}).findAll('tr') is not None:
                Team_stats_trs = soup.find('table', {'id':'team-stats-per_game'}).findAll('tr')
                for Team_stats_tr in Team_stats_trs:
                    thhead = Team_stats_tr.find('th',{'scope': 'row'})
                    sheet = []
                    if thhead is not None:
                        Team = Team_stats_tr.find('td',{'data-stat': 'team_name'}).text.strip()
                        if Team != "League Average":
                            G = Team_stats_tr.find('td',{'data-stat': 'g'}).text.strip()
                            MP = Team_stats_tr.find('td',{'data-stat': 'mp'}).text.strip()
                            FG = Team_stats_tr.find('td',{'data-stat': 'fg'}).text.strip()
                            FGA = Team_stats_tr.find('td',{'data-stat': 'fga'}).text.strip()
                            FGpercent = Team_stats_tr.find('td',{'data-stat': 'fg_pct'}).text.strip()
                            ThreeP = Team_stats_tr.find('td',{'data-stat': 'fg3'}).text.strip()
                            ThreePA = Team_stats_tr.find('td',{'data-stat': 'fg3a'}).text.strip()
                            ThreePpercent = Team_stats_tr.find('td',{'data-stat': 'fg3_pct'}).text.strip()
                            TwoP = Team_stats_tr.find('td',{'data-stat': 'fg2'}).text.strip()
                            TwoPA = Team_stats_tr.find('td',{'data-stat': 'fg2a'}).text.strip()
                            TwoPpercent = Team_stats_tr.find('td',{'data-stat': 'fg2_pct'}).text.strip()
                            FT = Team_stats_tr.find('td',{'data-stat': 'ft'}).text.strip()
                            FTA = Team_stats_tr.find('td',{'data-stat': 'fta'}).text.strip()
                            FTpercent = Team_stats_tr.find('td',{'data-stat': 'ft_pct'}).text.strip()
                            ORB = Team_stats_tr.find('td',{'data-stat': 'orb'}).text.strip()
                            DRB = Team_stats_tr.find('td',{'data-stat': 'drb'}).text.strip()
                            TRB = Team_stats_tr.find('td',{'data-stat': 'trb'}).text.strip()
                            AST = Team_stats_tr.find('td',{'data-stat': 'ast'}).text.strip()
                            STL = Team_stats_tr.find('td',{'data-stat': 'stl'}).text.strip()
                            BLK = Team_stats_tr.find('td',{'data-stat': 'blk'}).text.strip()
                            TOV = Team_stats_tr.find('td',{'data-stat': 'tov'}).text.strip()
                            PF = Team_stats_tr.find('td',{'data-stat': 'pf'}).text.strip()
                            PTS = Team_stats_tr.find('td',{'data-stat': 'pts'}).text.strip()

                            # Confirm the Win, Loss, and Conference
                            e_w_confirm = self.check_E_W(E_W_teams, Team)
                            if e_w_confirm != "error":
                                W = e_w_confirm['W']
                                L = e_w_confirm['L']
                                Conf = e_w_confirm['index']
                                
                            # append ALL team data
                            sheet.append(self.rk)
                            sheet.append(Team)
                            sheet.append(Year)
                            sheet.append(G)

                            sheet.append(W)
                            sheet.append(L)
                            sheet.append(Conf)

                            sheet.append(MP)
                            sheet.append(FG)
                            sheet.append(FGA)
                            sheet.append(FGpercent)
                            sheet.append(ThreeP)
                            sheet.append(ThreePA)
                            sheet.append(ThreePpercent)
                            sheet.append(TwoP)
                            sheet.append(TwoPA)
                            sheet.append(TwoPpercent)
                            sheet.append(FT)
                            sheet.append(FTA)
                            sheet.append(FTpercent)
                            sheet.append(ORB)
                            sheet.append(DRB)
                            sheet.append(AST)
                            sheet.append(STL)
                            sheet.append(BLK)
                            sheet.append(TOV)
                            sheet.append(PF)
                            sheet.append(PTS)
                            self.rk += 1
                            print(sheet)
                            nbawriter.writerow(sheet)
                            print("--------------------------------------------------------")
                            print(" ")
                            

        print("--------------------------------------------------------")
        print(" ")

        # move to next page to scrape if under year boundries
        if self.year <= (end_year-1):
            self.runcount += 1
            self.year += 1
            url = "https://www.basketball-reference.com/leagues/NBA_" + str(self.year) + ".html"
            self.getyearteamdata(url)

    def check_E_W(self, E_W_teams, teamname):
        t_name = "error"
        for E_W_team in E_W_teams:
            if E_W_team['teamname'] == teamname.replace("*",""):
                t_name = E_W_team
                break
        return t_name
NBAscrapy()

--------------year: 1980 | row: 1-----------------
[1, 'San Antonio Spurs*', 1980, '82', '41', '41', 'East', '240.9', '47.0', '94.4', '.498', '0.6', '2.5', '.252', '46.4', '91.9', '.505', '24.7', '30.8', '.801', '14.1', '30.7', '28.4', '9.4', '4.1', '19.4', '25.6', '119.4']
--------------------------------------------------------
 
[2, 'Los Angeles Lakers*', 1980, '82', '60', '22', 'West', '242.4', '47.5', '89.9', '.529', '0.2', '1.2', '.200', '47.3', '88.6', '.534', '19.8', '25.5', '.775', '13.2', '32.4', '29.4', '9.4', '6.7', '20.0', '21.8', '115.1']
--------------------------------------------------------
 
[3, 'Cleveland Cavaliers', 1980, '82', '37', '45', 'East', '243.0', '46.5', '98.1', '.474', '0.4', '2.3', '.193', '46.0', '95.8', '.481', '20.8', '26.9', '.772', '15.9', '29.0', '25.7', '9.3', '4.2', '16.7', '23.6', '114.1']
--------------------------------------------------------
 
[4, 'New York Knicks', 1980, '82', '39', '43', 'East', '241.2', '46.4', '93.6', '.496', '0.5', '2.

[1, 'Denver Nuggets*', 1982, '82', '46', '36', 'West', '241.8', '48.5', '93.4', '.520', '0.5', '1.8', '.268', '48.0', '91.5', '.525', '28.9', '36.3', '.796', '14.0', '29.8', '27.7', '8.1', '4.5', '17.9', '26.0', '126.5']
--------------------------------------------------------
 
[2, 'Los Angeles Lakers*', 1982, '82', '57', '25', 'West', '242.1', '47.8', '92.5', '.517', '0.2', '1.1', '.138', '47.6', '91.4', '.521', '18.9', '26.4', '.717', '15.3', '30.5', '28.7', '10.3', '6.3', '17.9', '24.4', '114.6']
--------------------------------------------------------
 
[3, 'San Antonio Spurs*', 1982, '82', '48', '34', 'West', '240.9', '45.1', '92.8', '.486', '0.8', '3.1', '.254', '44.3', '89.8', '.494', '22.1', '28.5', '.776', '15.3', '30.9', '27.5', '7.3', '6.8', '15.8', '27.0', '113.1']
--------------------------------------------------------
 
[4, 'Boston Celtics*', 1982, '82', '63', '19', 'East', '241.2', '44.6', '89.4', '.499', '0.6', '2.2', '.266', '44.0', '87.2', '.505', '22.2', '30.0', '.

[1, 'Denver Nuggets*', 1984, '82', '38', '44', 'West', '241.5', '48.0', '97.4', '.493', '0.9', '3.1', '.302', '47.0', '94.2', '.499', '26.8', '32.8', '.818', '13.8', '29.8', '30.3', '8.7', '4.3', '16.4', '27.8', '123.7']
--------------------------------------------------------
 
[2, 'San Antonio Spurs', 1984, '82', '37', '45', 'West', '243.7', '47.7', '94.2', '.506', '1.0', '3.2', '.300', '46.7', '91.0', '.514', '24.0', '31.8', '.755', '15.0', '30.8', '28.8', '8.4', '6.0', '17.6', '26.2', '120.3']
--------------------------------------------------------
 
[3, 'Detroit Pistons*', 1984, '82', '49', '33', 'East', '242.7', '46.3', '96.5', '.480', '0.4', '1.7', '.227', '45.9', '94.7', '.485', '24.1', '31.1', '.775', '17.4', '29.7', '27.5', '8.5', '5.1', '16.0', '26.5', '117.1']
--------------------------------------------------------
 
[4, 'Los Angeles Lakers*', 1984, '82', '54', '28', 'West', '241.2', '47.0', '88.4', '.532', '0.7', '2.8', '.257', '46.3', '85.7', '.540', '20.9', '27.7', '.7

[1, 'Los Angeles Lakers*', 1986, '82', '62', '20', 'West', '241.8', '46.8', '89.5', '.522', '1.7', '5.0', '.337', '45.1', '84.6', '.533', '22.1', '28.4', '.778', '13.4', '31.2', '29.7', '8.5', '5.1', '17.9', '24.8', '117.3']
--------------------------------------------------------
 
[2, 'Dallas Mavericks*', 1986, '82', '44', '38', 'West', '241.5', '44.3', '88.5', '.501', '1.7', '5.4', '.316', '42.6', '83.0', '.513', '25.0', '32.2', '.776', '12.9', '29.9', '25.7', '7.4', '4.5', '15.7', '21.1', '115.3']
--------------------------------------------------------
 
[3, 'Portland Trail Blazers*', 1986, '82', '40', '42', 'West', '242.1', '44.0', '88.8', '.496', '0.9', '3.4', '.269', '43.1', '85.4', '.505', '26.1', '34.1', '.765', '14.1', '28.2', '26.6', '10.5', '4.3', '18.6', '26.9', '115.1']
--------------------------------------------------------
 
[4, 'Denver Nuggets*', 1986, '82', '47', '35', 'West', '240.9', '45.2', '96.0', '.471', '0.9', '3.7', '.233', '44.3', '92.2', '.480', '23.5', '29

[1, 'Denver Nuggets*', 1988, '82', '54', '28', 'West', '241.2', '46.0', '97.1', '.474', '2.3', '6.9', '.342', '43.6', '90.2', '.484', '22.5', '27.9', '.804', '14.2', '29.8', '28.0', '10.1', '4.9', '14.5', '24.2', '116.7']
--------------------------------------------------------
 
[2, 'Portland Trail Blazers*', 1988, '82', '53', '29', 'West', '240.6', '44.6', '91.0', '.491', '1.4', '4.6', '.308', '43.2', '86.3', '.501', '25.4', '32.9', '.770', '15.3', '30.4', '28.1', '8.9', '4.2', '16.5', '25.5', '116.1']
--------------------------------------------------------
 
[3, 'Boston Celtics*', 1988, '82', '57', '25', 'East', '241.2', '43.9', '84.2', '.521', '3.3', '8.6', '.384', '40.6', '75.6', '.537', '22.5', '28.0', '.803', '11.3', '29.8', '29.9', '7.6', '5.1', '15.9', '22.0', '113.6']
--------------------------------------------------------
 
[4, 'San Antonio Spurs*', 1988, '82', '31', '51', 'West', '241.5', '45.2', '92.2', '.490', '1.6', '5.0', '.323', '43.6', '87.2', '.500', '21.6', '29.4'

...

[1, 'Golden State Warriors*', 2016, '82', '73', '9', 'West', '242.4', '42.5', '87.3', '.487', '13.1', '31.6', '.416', '29.4', '55.7', '.528', '16.7', '21.8', '.763', '10.0', '36.2', '28.9', '8.4', '6.1', '15.2', '20.7', '114.9']
--------------------------------------------------------
 
[2, 'Oklahoma City Thunder*', 2016, '82', '55', '27', 'West', '241.8', '41.1', '86.4', '.476', '8.3', '23.7', '.349', '32.9', '62.6', '.524', '19.7', '25.2', '.782', '13.1', '35.6', '23.0', '7.4', '5.9', '15.9', '20.6', '110.2']
--------------------------------------------------------
 
[3, 'Sacramento Kings', 2016, '82', '33', '49', 'West', '241.5', '40.0', '86.4', '.464', '8.0', '22.4', '.359', '32.0', '64.0', '.500', '18.5', '25.5', '.725', '10.6', '33.7', '24.5', '8.9', '4.5', '16.2', '20.4', '106.6']
--------------------------------------------------------
 
[4, 'Houston Rockets*', 2016, '82', '41', '41', 'West', '241.8', '37.7', '83.5', '.452', '10.7', '30.9', '.347', '27.0', '52.6', '.514', '20.4

[1, 'Golden State Warriors*', 2017, '82', '67', '15', 'West', '241.2', '43.1', '87.1', '.495', '12.0', '31.2', '.383', '31.1', '55.8', '.557', '17.8', '22.6', '.788', '9.4', '35.0', '30.4', '9.6', '6.8', '14.8', '19.3', '115.9']
--------------------------------------------------------
 
[2, 'Houston Rockets*', 2017, '82', '55', '27', 'West', '241.2', '40.3', '87.2', '.462', '14.4', '40.3', '.357', '25.9', '46.9', '.552', '20.3', '26.5', '.766', '10.9', '33.5', '25.2', '8.2', '4.3', '15.1', '19.9', '115.3']
--------------------------------------------------------
 
[3, 'Denver Nuggets', 2017, '82', '40', '42', 'West', '240.9', '41.2', '87.7', '.469', '10.6', '28.8', '.368', '30.6', '58.9', '.519', '18.7', '24.2', '.774', '11.8', '34.6', '25.3', '6.9', '3.9', '15.0', '19.1', '111.7']
--------------------------------------------------------
 
[4, 'Cleveland Cavaliers*', 2017, '82', '51', '31', 'East', '242.4', '39.9', '84.9', '.470', '13.0', '33.9', '.384', '26.9', '51.0', '.528', '17.5',

[1, 'Golden State Warriors*', 2018, '82', '58', '24', 'West', '240.6', '42.8', '85.1', '.503', '11.3', '28.9', '.391', '31.5', '56.2', '.560', '16.6', '20.3', '.815', '8.4', '35.1', '29.3', '8.0', '7.5', '15.5', '19.6', '113.5']
--------------------------------------------------------
 
[2, 'Houston Rockets*', 2018, '82', '65', '17', 'West', '240.9', '38.7', '84.2', '.460', '15.3', '42.3', '.362', '23.4', '41.9', '.558', '19.6', '25.1', '.781', '9.0', '34.5', '21.5', '8.5', '4.8', '13.8', '19.5', '112.4']
--------------------------------------------------------
 
[3, 'New Orleans Pelicans*', 2018, '82', '48', '34', 'West', '243.4', '42.7', '88.3', '.483', '10.2', '28.2', '.362', '32.5', '60.1', '.540', '16.1', '20.9', '.772', '8.7', '35.7', '26.8', '8.0', '5.9', '14.9', '19.1', '111.7']
--------------------------------------------------------
 
[4, 'Toronto Raptors*', 2018, '82', '59', '23', 'East', '241.8', '41.3', '87.4', '.472', '11.8', '33.0', '.358', '29.5', '54.4', '.541', '17.3'

[1, 'Milwaukee Bucks*', 2019, '82', '60', '22', 'East', '241.2', '43.4', '91.1', '.476', '13.5', '38.2', '.353', '29.9', '52.9', '.565', '17.9', '23.2', '.773', '9.3', '40.4', '26.0', '7.5', '5.9', '13.9', '19.6', '118.1']
--------------------------------------------------------
 
[2, 'Golden State Warriors*', 2019, '82', '57', '25', 'West', '241.5', '44.0', '89.8', '.491', '13.3', '34.4', '.385', '30.8', '55.3', '.557', '16.3', '20.4', '.801', '9.7', '36.5', '29.4', '7.6', '6.4', '14.3', '21.4', '117.7']
--------------------------------------------------------
 
[3, 'New Orleans Pelicans', 2019, '82', '33', '49', 'West', '240.9', '43.7', '92.2', '.473', '10.3', '29.9', '.344', '33.4', '62.4', '.536', '17.8', '23.4', '.761', '11.1', '36.2', '27.0', '7.4', '5.4', '14.8', '21.1', '115.4']
--------------------------------------------------------
 
[4, 'Philadelphia 76ers*', 2019, '82', '51', '31', 'East', '241.5', '41.5', '88.2', '.471', '10.8', '30.2', '.359', '30.7', '58.0', '.529', '21

[1, 'Dallas Mavericks', 2020, '67', '40', '27', 'West', '241.5', '41.6', '90.0', '.462', '15.3', '41.5', '.369', '26.3', '48.5', '.542', '17.9', '23.1', '.773', '10.6', '36.4', '24.5', '6.3', '5.0', '12.8', '19.0', '116.4']
--------------------------------------------------------
 
[2, 'Milwaukee Bucks*', 2020, '65', '53', '12', 'East', '240.8', '43.5', '91.2', '.477', '13.7', '38.6', '.356', '29.8', '52.6', '.567', '17.8', '24.0', '.742', '9.5', '42.2', '25.9', '7.4', '6.0', '14.9', '19.2', '118.6']
--------------------------------------------------------
 
[3, 'Houston Rockets', 2020, '64', '40', '24', 'West', '241.2', '41.1', '90.7', '.454', '15.4', '44.3', '.348', '25.7', '46.4', '.554', '20.5', '26.0', '.787', '10.4', '34.6', '21.5', '8.5', '5.1', '14.7', '21.6', '118.1']
--------------------------------------------------------
 
[4, 'Portland Trail Blazers', 2020, '66', '29', '37', 'West', '240.8', '41.9', '90.9', '.461', '12.6', '33.8', '.372', '29.3', '57.1', '.513', '17.3', '2